In [1]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit
from iminuit import cost
from scipy import stats
import sympy as sp 
from sympy import symbols, diff, lambdify, solve 

In [2]:
data = np.genfromtxt('test_pend2.txt')
# x = data[:,0]
# y = data[:,1]
# yerr = data[:,2]

x = data[:,0]
y = data[:,1]


In [12]:
L,T = symbols('L T')

g = L* (2*sp.pi/T)**2

def error_prop(function,variables,errors):
    total_error = 0 
    for i in range(len(variables)):
        total_error += (diff(function,variables[i])**2)*(errors[i]**2)
    return sp.sqrt(total_error)


def resid_plot(y,yfit):
    plt.hist(y-yfit,bins=6)
    plt.show()
    
def gauss_pdf(x, mu, sigma) :
    """Gaussian"""
    return 1.0 / np.sqrt(2*np.pi* sigma**2) * np.exp( -0.5 * (x-mu)**2 / sigma**2)


def fit_funcT(x,a,b):
    return a*x+b
def fit_funcg(x,time):
    return x*(2*np.pi/time)**2

# def linearfit(x,y,function,guessederror,params):
#     cfit = cost.LeastSquares(x, y, yerr=guessederror, model=function)
#     m = Minuit(cfit, **params)
#     fit_params = m.migrad().parameters 
    

In [13]:
def fit(x,y):
    cfit = cost.LeastSquares(x, y,1,  model=fit_funcT)
    m = Minuit(cfit, a=1, b=1)
    m.migrad() # finds minimum of least_squares function 
    fit_params = m.values   
    a,b = fit_params['a'],fit_params['b']
    yfit = fit_funcT(x,a,b) 
    
    #residual histogram
    count, bins = np.histogram(y-yfit, bins=int(np.sqrt(len(y))))
    sy = np.sqrt(count)
    gaussfit = cost.LeastSquares(bins[:-1], count, sy, model=gauss_pdf)
    count = count[count>0]
    m = Minuit(gaussfit, mu=0, sigma=1)
    m.limits["sigma"] = (0, None)  # Enforce sigma > 0
    m.migrad()
    sigma = m.values['sigma']
    print( f"std for gaussian:  {sigma:.2f}")
    
    #Refit with new error on each meassurement
    cfit = cost.LeastSquares(x, y,sigma,  model=fit_funcT)
    m = Minuit(cfit, a=a, b=b)
    m.migrad() # finds minimum of least_squares function
    yfit = fit_funcT(x,*m.values)
    a,b = m.values 
    ae,be = m.errors
    print(f"a = {a:.4f} +- {ae:.4f}")
    print(f"b = {b:.2f} +- {be:.2f}")
    return m.errors, m.values
print(fit(x,y))

std for gaussian:  0.10
a = 7.5143 +- 0.0028
b = -0.08 +- 0.04
(<ErrorView a=0.002762130661383447 b=0.041062026927875014>, <ValueView a=7.514321461536414 b=-0.07851100003729908>)


In [ ]:
a = fit(x,y)


std for gaussian:  0.10
a = 7.5143 +- 0.0028
b = -0.08 +- 0.04


In [14]:
L1 = 90.2 
errorL2 = 0.1 
L2 = 22.1
errorL2 =  0.1
O,A = symbols('O A')


GAngle1 = 14.00 #Ball Towards RIGET
Gangle2 = 14.50 #Ball Away from RIGET
AngleE = 0.5 #deg

angle = np.tan(L2/L1) * 180/np.pi
Angle = sp.tan(O/A) * 180/np.pi 

func = error_prop(Angle,[O,A],[0.1,0.1])

Aerror = sp.lambdify([O,A],func)(O=L2,A=L1)

print(f"Measured angle through trig: {angle:.2f} +- {Aerror:.2f}")


sumerror = 1/Aerror**2 + 1/AngleE**2 + 1/AngleE**2
sumW = angle/Aerror**2 + GAngle1/AngleE**2 + Gangle2/AngleE**2 
angleW = sumW/sumerror

print(f"Weighted average: {angleW:.2f} +- {np.sqrt(1/sumerror):.2f}")

Measured angle through trig: 14.33 +- 0.07
Weighted average: 14.32 +- 0.07
